In [ ]:
import cv2
import numpy as np

In [ ]:
prototxtPath = r"Downloads\deploy.prototxt"
weightsPath = r"Downloads\res10_300x300_ssd_iter_140000.caffemodel"
faceNet = cv2.dnn.readNet(prototxtPath, weightsPath)
labels_dict={0:'NO MASK',1:'MASK'}
color_dict={0:(0,0,255),1:(0,255,0)}

In [ ]:
capture = cv2.VideoCapture(0)
data_without=[]
while True:
  flag,img=capture.read()
  if flag:
    h, w = img.shape[:2]
      
    # Preprocess the frame: resize, normalize, and create a blob
    blob = cv2.dnn.blobFromImage(img, 1.0, (300, 300), (104.0, 177.0, 123.0))
    faceNet.setInput(blob)

    # Perform face detection
    detections = faceNet.forward()

    for i in range(0, detections.shape[2]):
        confidence = detections[0, 0, i, 2]

        # Filter detections with confidence > 0.5
        if confidence > 0.5:
            # Get bounding box coordinates
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")

            # Draw a rectangle around the detected face
            cv2.rectangle(img, (startX, startY), (endX, endY), (255, 0, 255), 4)

            # Extract and preprocess the face region
            face = img[startY:endY, startX:endX]
            face = cv2.resize(face, (100, 100))
            data_without.append(face)
            
            print(f"Captured faces: {len(data_without)}")


    cv2.imshow('OUTPUT',img)
    if cv2.waitKey(1) == 27 or len(data_without)>=400:
      break
capture.release()
cv2.destroyAllWindows()

In [ ]:
import matplotlib.image as mpimg

for i in range(len(data_without)):
    mpimg.imsave(f"dataset/without_mask/{i}.png", data_without[i])
